In [5]:
from bm_features import XTTSFeatureExtractor, XTTSProcessor, XTTSTokenizer
from datasets import load_dataset
import datasets
import torch

In [4]:
1+1

2

In [6]:
dataset = load_dataset("oza75/bambara-tts", "denoised")
dataset = dataset.cast_column("audio", datasets.Audio(sampling_rate=22050)).rename_column('bambara', 'text')
dataset

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'french', 'duration', 'speaker_embeddings', 'speaker_id'],
        num_rows: 30765
    })
})

In [7]:
feature_extractor = XTTSFeatureExtractor("../mel_stats.pth", sampling_rate=22050, max_samples=221000)
tokenizer = XTTSTokenizer.from_pretrained("openai-community/gpt2-large")
processor = XTTSProcessor(feature_extractor, tokenizer)

batch = processor(dataset['train'][:2])
batch

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'XTTSTokenizer'.


{'audio': [{'path': None,
   'array': array([ 0.00099032,  0.00157957,  0.00133549, ..., -0.00630357,
          -0.00620323, -0.00616569]),
   'sampling_rate': 22050},
  {'path': None,
   'array': array([ 0.00011191,  0.00036082,  0.00023281, ..., -0.00214735,
          -0.0011257 , -0.00175181]),
   'sampling_rate': 22050}],
 'text': ['Jigi, i bolo degunnen don wa ?', 'Dɔɔnin !'],
 'french': ['Jigi, es-tu occupé ?', 'Un peu! '],
 'duration': [2.645986394557823, 0.8740136054421769],
 'speaker_embeddings': tensor([[ -2.5645, -20.9284,  69.9060,  ...,  22.0783,  42.1153,  19.8596],
         [ -2.5297,  57.3677, -24.4568,  ...,  72.7989, 108.3561,  71.1548]]),
 'speaker_id': [22, 27],
 'wav_mels': tensor([[[0.8004, 0.9520, 1.2643,  ..., 1.6425, 1.6425, 1.6425],
          [1.0748, 1.2971, 1.7636,  ..., 1.8926, 1.8926, 1.8926],
          [1.6542, 1.9314, 2.3495,  ..., 2.4791, 2.4791, 2.4791],
          ...,
          [1.2285, 1.2285, 1.2285,  ..., 1.2285, 1.2285, 1.2285],
          [1.2226,

In [4]:
batch['wav_mels'].shape

torch.Size([2, 80, 864])

In [4]:
from bm_models import Xtts, XttsConfig

model_config = XttsConfig(
    gpt_number_text_tokens=processor.tokenizer.vocab_size,
    gpt_start_text_token=processor.tokenizer.convert_tokens_to_ids(processor.tokenizer.START_TOKEN),
    gpt_stop_text_token=processor.tokenizer.convert_tokens_to_ids(processor.tokenizer.STOP_TOKEN),
)
model = Xtts(model_config)
model

Xtts(
  (gpt_wrapper): GPTWrapper(
    (text_emb): Embedding(50257, 1024)
    (mel_emb): Embedding(605, 1024)
    (gpt): GPT2Model(
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-29): 30 x GPT2Block(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
    (text_pos_emb): LearnedPositionEmbeddings(
      (emb): Embedding(402, 1024)
    )
    (mel_pos_emb): LearnedPos

In [7]:
from transformers import GPT2Model, GPT2Config

model_config = GPT2Config(
    n_layer=30,
    n_head=16,
    n_embd=1024,
    n_positions=402 + 605 + 70,
    n_ctx=402 + 605 + 70,
)
gpt_model = GPT2Model.from_pretrained("openai-community/gpt2-medium", config=model_config, ignore_mismatched_sizes=True)
gpt_model

Some weights of GPT2Model were not initialized from the model checkpoint at openai-community/gpt2-medium and are newly initialized: ['h.24.attn.c_attn.bias', 'h.24.attn.c_attn.weight', 'h.24.attn.c_proj.bias', 'h.24.attn.c_proj.weight', 'h.24.ln_1.bias', 'h.24.ln_1.weight', 'h.24.ln_2.bias', 'h.24.ln_2.weight', 'h.24.mlp.c_fc.bias', 'h.24.mlp.c_fc.weight', 'h.24.mlp.c_proj.bias', 'h.24.mlp.c_proj.weight', 'h.25.attn.c_attn.bias', 'h.25.attn.c_attn.weight', 'h.25.attn.c_proj.bias', 'h.25.attn.c_proj.weight', 'h.25.ln_1.bias', 'h.25.ln_1.weight', 'h.25.ln_2.bias', 'h.25.ln_2.weight', 'h.25.mlp.c_fc.bias', 'h.25.mlp.c_fc.weight', 'h.25.mlp.c_proj.bias', 'h.25.mlp.c_proj.weight', 'h.26.attn.c_attn.bias', 'h.26.attn.c_attn.weight', 'h.26.attn.c_proj.bias', 'h.26.attn.c_proj.weight', 'h.26.ln_1.bias', 'h.26.ln_1.weight', 'h.26.ln_2.bias', 'h.26.ln_2.weight', 'h.26.mlp.c_fc.bias', 'h.26.mlp.c_fc.weight', 'h.26.mlp.c_proj.bias', 'h.26.mlp.c_proj.weight', 'h.27.attn.c_attn.bias', 'h.27.attn.c_a

GPT2Model(
  (wte): Embedding(50257, 1024)
  (wpe): Embedding(1077, 1024)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-29): 30 x GPT2Block(
      (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
)

In [10]:
import functools

def null_position_embeddings(range, dim):
    return torch.zeros((range.shape[0], range.shape[1], dim), device=range.device)

del gpt_model.wpe
del gpt_model.wte
gpt_model.wpe = functools.partial(null_position_embeddings, dim=1024)
gpt_model

GPT2Model(
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-29): 30 x GPT2Block(
      (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
)

In [12]:
model.gpt_wrapper.gpt.load_state_dict(gpt_model.state_dict())

<All keys matched successfully>

In [5]:
model(
    input_ids=batch['text_tokens'],
    label_ids=batch['text_tokens'],  # same as text_tokens
    text_lengths=batch['text_lengths'],
    cond_mels=batch['cond_mels'],
    cond_idxs=batch['cond_idxs'],
    cond_lens=batch['cond_len'],
    wav_mels=batch['wav_mels'],
    wav_mel_attention_masks=batch['wav_mel_attention_masks'],
    wav_lengths=batch['wav_lengths'],
)

IndexError: index out of range in self

In [9]:
def count_parameters(model):
    # Total number of parameters
    total_params = sum(p.numel() for p in model.parameters())

    # Total number of trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Total number of non-trainable parameters
    non_trainable_params = total_params - trainable_params

    print(f"Total number of parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Non-trainable parameters: {non_trainable_params:,}")


count_parameters(model)

Total number of parameters: 562,297,064
Trainable parameters: 562,297,064
Non-trainable parameters: 0


In [10]:
count_parameters(model.gpt.gpt)

Total number of parameters: 377,888,768
Trainable parameters: 377,888,768
Non-trainable parameters: 0


In [12]:
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth

--2024-06-08 03:01:01--  https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth
Resolving coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)... 54.203.228.158, 52.32.4.234, 54.186.175.150
Connecting to coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)|54.203.228.158|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth [following]
--2024-06-08 03:01:01--  https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth
Resolving huggingface.co (huggingface.co)... 3.161.213.58, 3.161.213.25, 3.161.213.110, ...
Connecting to huggingface.co (huggingface.co)|3.161.213.58|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/3e/73/3e73be38940b0ce7d10274a3412defcd0d3c9ea879d08bc8761d46cf6bbde3fa/c7ea20001c6a0a841c77e252d8409f6a74fb423e79b3206a0771ba5989776187?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.pth%3B+filename%3D

In [8]:
from bm_models import XttsConfig, Xtts

model = Xtts.from_pretrained("./bm_xtts")
model

Some weights of the model checkpoint at ./bm_xtts were not used when initializing Xtts: ['gpt_wrapper.conditioning_perceiver.norm.weight']
- This IS expected if you are initializing Xtts from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Xtts from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Xtts were not initialized from the model checkpoint at ./bm_xtts and are newly initialized: ['gpt_wrapper.conditioning_perceiver.norm.gamma']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Xtts(
  (gpt_wrapper): GPTWrapper(
    (text_emb): Embedding(50257, 1024)
    (mel_emb): Embedding(605, 1024)
    (gpt): GPT2Model(
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-29): 30 x GPT2Block(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
    (text_pos_emb): LearnedPositionEmbeddings(
      (emb): Embedding(402, 1024)
    )
    (mel_pos_emb): LearnedPos

In [9]:
pretrained_states = torch.load("model.pth")
pretrained_states

{'config': XttsConfig(output_path='output', logger_uri=None, run_name='run', project_name=None, run_description='🐸Coqui trainer run.', print_step=25, plot_step=100, model_param_stats=False, wandb_entity=None, dashboard_logger='tensorboard', save_on_interrupt=True, log_model_step=None, save_step=10000, save_n_checkpoints=5, save_checkpoints=True, save_all_best=False, save_best_after=10000, target_loss=None, print_eval=False, test_delay_epochs=0, run_eval=True, run_eval_steps=None, distributed_backend='nccl', distributed_url='tcp://localhost:54321', mixed_precision=False, precision='fp16', epochs=1000, batch_size=32, eval_batch_size=16, grad_clip=0.0, scheduler_after_epoch=True, lr=0.001, optimizer='radam', optimizer_params=None, lr_scheduler=None, lr_scheduler_params={}, use_grad_scaler=False, allow_tf32=False, cudnn_enable=True, cudnn_deterministic=False, cudnn_benchmark=False, training_seed=54321, model='xtts', num_loader_workers=0, num_eval_loader_workers=0, use_noise_augment=False, 

In [10]:
perceiver_weights = {key.replace('gpt.conditioning_perceiver.', ''): pretrained_states['model'][key] for key in pretrained_states['model'].keys()
               if "gpt.conditioning_perceiver" in key}
cond_encoder_weights = {key.replace('gpt.conditioning_encoder.', ''): pretrained_states['model'][key] for key in pretrained_states['model'].keys()
               if "gpt.conditioning_encoder" in key}
cond_encoder_drop_weights = {key.replace('gpt.conditioning_dropout.', ''): pretrained_states['model'][key] for key in pretrained_states['model'].keys()
               if "gpt.conditioning_dropout" in key}

model.gpt_wrapper.conditioning_perceiver.load_state_dict(perceiver_weights)
model.gpt_wrapper.conditioning_encoder.load_state_dict(cond_encoder_weights)
model.gpt_wrapper.conditioning_dropout.load_state_dict(cond_encoder_drop_weights)

<All keys matched successfully>

In [15]:
gpt_weights = {key.replace('gpt.gpt.', ''): pretrained_states['model'][key] for key in pretrained_states['model'].keys()
               if "gpt.gpt" in key}
gpt_mel_pos_emb = {key.replace('gpt.mel_pos_embedding.', ''): pretrained_states['model'][key] for key in
                   pretrained_states['model'].keys() if "gpt.mel_pos_embedding" in key}
gpt_text_pos_emb = {key.replace('gpt.text_pos_embedding.', ''): pretrained_states['model'][key] for key in
                    pretrained_states['model'].keys() if "gpt.text_pos_embedding" in key}
model.gpt.gpt.load_state_dict(gpt_weights)
model.gpt.text_pos_embedding.load_state_dict(gpt_text_pos_emb)
model.gpt.mel_pos_embedding.load_state_dict(gpt_mel_pos_emb)

<All keys matched successfully>

In [15]:
model.state_dict().keys()

odict_keys(['gpt_wrapper.text_emb.weight', 'gpt_wrapper.mel_emb.weight', 'gpt_wrapper.gpt.h.0.ln_1.weight', 'gpt_wrapper.gpt.h.0.ln_1.bias', 'gpt_wrapper.gpt.h.0.attn.c_attn.weight', 'gpt_wrapper.gpt.h.0.attn.c_attn.bias', 'gpt_wrapper.gpt.h.0.attn.c_proj.weight', 'gpt_wrapper.gpt.h.0.attn.c_proj.bias', 'gpt_wrapper.gpt.h.0.ln_2.weight', 'gpt_wrapper.gpt.h.0.ln_2.bias', 'gpt_wrapper.gpt.h.0.mlp.c_fc.weight', 'gpt_wrapper.gpt.h.0.mlp.c_fc.bias', 'gpt_wrapper.gpt.h.0.mlp.c_proj.weight', 'gpt_wrapper.gpt.h.0.mlp.c_proj.bias', 'gpt_wrapper.gpt.h.1.ln_1.weight', 'gpt_wrapper.gpt.h.1.ln_1.bias', 'gpt_wrapper.gpt.h.1.attn.c_attn.weight', 'gpt_wrapper.gpt.h.1.attn.c_attn.bias', 'gpt_wrapper.gpt.h.1.attn.c_proj.weight', 'gpt_wrapper.gpt.h.1.attn.c_proj.bias', 'gpt_wrapper.gpt.h.1.ln_2.weight', 'gpt_wrapper.gpt.h.1.ln_2.bias', 'gpt_wrapper.gpt.h.1.mlp.c_fc.weight', 'gpt_wrapper.gpt.h.1.mlp.c_fc.bias', 'gpt_wrapper.gpt.h.1.mlp.c_proj.weight', 'gpt_wrapper.gpt.h.1.mlp.c_proj.bias', 'gpt_wrapper.gp

In [17]:
from bm_utils.vq_vae import BMSpeechVQVAE

vq_model = BMSpeechVQVAE.from_pretrained("oza75/bambara-vqvae")
model.vq_vae.load_state_dict(vq_model.state_dict())

<All keys matched successfully>

In [11]:
hifigan_decoder = {key.replace('hifigan_decoder.', ''): pretrained_states['model'][key] for key in
                   pretrained_states['model'].keys() if "hifigan_decoder." in key}
model.hifigan_decoder.load_state_dict(hifigan_decoder)

<All keys matched successfully>

In [18]:
model.save_pretrained("./bm_xtts")

[2024-06-14 13:47:05,978] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to mps (auto detect)


W0614 13:47:06.079000 8211254272 torch/distributed/elastic/multiprocessing/redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.


In [7]:
from bm_utils.dvae import DiscreteVAE

dvae = DiscreteVAE(
    channels=80,
    normalization=None,
    positional_dims=1,
    num_tokens=1024,
    codebook_dim=512,
    hidden_dim=512,
    num_resnet_blocks=3,
    kernel_size=3,
    num_layers=2,
    use_transposed_convs=False,
)
dvae

DiscreteVAE(
  (discrete_loss): DiscretizationLoss()
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv1d(80, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(512, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): ReLU()
    )
    (2): ResBlock(
      (net): Sequential(
        (0): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
        (2): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (3): ReLU()
        (4): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
      )
    )
    (3): ResBlock(
      (net): Sequential(
        (0): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU()
        (2): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (3): ReLU()
        (4): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
      )
    )
    (4): ResBlock(
      (net): Sequential(
        (0):

In [8]:
!wget https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth

--2024-06-08 03:08:34--  https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth
Resolving coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)... 54.203.228.158, 52.32.4.234, 54.186.175.150
Connecting to coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)|54.203.228.158|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://huggingface.co/coqui/XTTS-v2/resolve/main/dvae.pth [following]
--2024-06-08 03:08:34--  https://huggingface.co/coqui/XTTS-v2/resolve/main/dvae.pth
Resolving huggingface.co (huggingface.co)... 3.161.213.110, 3.161.213.58, 3.161.213.25, ...
Connecting to huggingface.co (huggingface.co)|3.161.213.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/3e/73/3e73be38940b0ce7d10274a3412defcd0d3c9ea879d08bc8761d46cf6bbde3fa/b29bc227d410d4991e0a8c09b858f77415013eeb9fba9650258e96095557d97a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27dvae.pth%3B+filename%3D%22

In [9]:
dvae_states = torch.load('dvae.pth')
dvae.eval()
dvae.load_state_dict(dvae_states, strict=False)

_IncompatibleKeys(missing_keys=['discrete_loss.accumulator_index', 'discrete_loss.accumulator_filled', 'discrete_loss.accumulator'], unexpected_keys=[])

In [11]:
dvae_codes = dvae.get_codebook_indices(batch['wav_mels'])
dvae_codes

embed_ind:  tensor([ 984,  588,  785,  590,   66,   11,  396,   11,   11,   11,  353,  448,
         644,   21,  509,  620,  651,  645,  891,  447,  283,  948, 1011, 1011,
        1011, 1011, 1006, 1008,  302,  340,  424,  846,  598,  581,   30,  405,
         424,  312,  389,  464,  226,  619,  267,   66,  173,  792,  363,  209,
          22,  356,   22,  728,  559,    9,  538,  538,  538,  298,  992, 1006,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
        1011, 1011, 1011, 10

tensor([[ 984,  588,  785,  590,   66,   11,  396,   11,   11,   11,  353,  448,
          644,   21,  509,  620,  651,  645,  891,  447,  283,  948, 1011, 1011,
         1011, 1011, 1006, 1008,  302,  340,  424,  846,  598,  581,   30,  405,
          424,  312,  389,  464,  226,  619,  267,   66,  173,  792,  363,  209,
           22,  356,   22,  728,  559,    9,  538,  538,  538,  298,  992, 1006,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011, 1011,
         1011, 1011, 1011, 1

In [25]:
dvae_codes.shape

torch.Size([2, 216])

In [12]:
B, n_mels, time_steps = batch['wav_mels'].shape

In [13]:
vq_indices = model.vq_vae.get_codebook_indices(batch['wav_mels'].view(B, 1, n_mels, time_steps))

In [14]:
vq_indices

(tensor([[454, 389, 389, 389, 466, 466,  72, 447,  32, 448, 399,  64, 483, 234,
          140,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,
           72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,
           72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72, 462,
          220, 162, 153,  72,  72, 399,  96, 369, 222,  32,  32, 220,  49,  72,
           72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,
           72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,  72,
           72,  72,  72,  72,  72,  72,  72,  72,  72, 119, 409, 476, 302, 166,
          135, 197, 166, 399, 209, 369, 131, 474, 131, 456, 490, 166, 166, 166,
          166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166,
          166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166, 166,
          166, 166, 166, 166, 166, 166, 166, 346, 409, 287, 118,  65, 476, 334,
          166, 399,  90, 467, 265, 347, 

In [15]:
vq_indices[0].shape

torch.Size([2, 270])

In [3]:
import torch.nn.functional as F
import torch

t = torch.tensor([0,1,2,4])
F.pad(t, (1, 0), value=120)

tensor([120,   0,   1,   2,   4])

In [5]:
torch.ones(10, 5, dtype=torch.bool)

tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])